In [1]:
import genjax
import jax

key = jax.random.PRNGKey(314159)

console = genjax.pretty()

In [2]:
genjax.adev.registry

{
    <class 'genjax._src.generative_functions.distributions.scipy.normal._Normal'>: <class 'genjax._src.generative_functions.distributions.adev_registry.ADEVPrimNormal'>,
    <class 'genjax._src.generative_functions.distributions.scipy.bernoulli._Bernoulli'>: <class 'genjax._src.generative_functions.distributions.adev_registry.ADEVPrimBernoulli'>,
    <class 'genjax._src.generative_functions.distributions.scipy.poisson._Poisson'>: <class 'genjax._src.generative_functions.distributions.adev_registry.ADEVPrimPoisson'>
}

In [12]:
@genjax.gen
def model():
    x = genjax.Normal(0.0, 1.0) @ "x"
    return x**2

In [13]:
model

BuiltinGenerativeFunction
└── source
    └── PytreeClosure
        ├── callable
        │   └── <function model>
        └── environment
            └── list

In [14]:
prog = genjax.adev.adev(model)

In [15]:
key, v = prog.simulate(key, ())
v

Array(1.8149817, dtype=float32)